# __Implementing CNN (Convulusion Neural Network) from scratch__
### without machine learning library and with numpy and pandas only


##### imports

In [ ]:
# import library 
import os 
import numpy as np 
import struct 
import gzip
import matplotlib.pyplot as plt
from math import sqrt

##### const variable

In [3]:
# const
DATASET_TYPE = "byclass"
RAW_DIR      = "gzip/"
UNPACK_DIR   = f"unpacked_{DATASET_TYPE}"
NPY_DIR      = "datasets_npy"

##### Reading, Storing data


In [4]:

def read_idx(file_name):
    with open(file_name, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        data = np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    return data

def load_emnist_byclass(path=UNPACK_DIR, dataset_type = DATASET_TYPE, np_dataset=NPY_DIR):
    if os.path.exists(np_dataset):
        return  

    # files are inside gzip.zip -> extract first
    X_train = read_idx(os.path.join(path, f"emnist-{dataset_type}-train-images-idx3-ubyte")) 
    y_train = read_idx(os.path.join(path, f"emnist-{dataset_type}-train-labels-idx1-ubyte")) 
    X_test  = read_idx(os.path.join(path, f"emnist-{dataset_type}-test-images-idx3-ubyte")) 
    y_test  = read_idx(os.path.join(path, f"emnist-{dataset_type}-test-labels-idx1-ubyte")) 
    
    save_binary(X_train, y_train, X_test, y_test)

def save_binary(X_train, y_train, X_test, y_test, path=NPY_DIR):
    if os.path.exists(path):
        return 
    # create folder 
    os.makedirs(path)

    # save ubyte to binary format 
    np.save(os.path.join(path, "X_train.npy"), X_train.astype("float32") / 255)
    np.save(os.path.join(path, "Y_train.npy"), y_train.astype("float32"))
    np.save(os.path.join(path, "X_test.npy"), X_test.astype("float32") / 255)
    np.save(os.path.join(path, "Y_test.npy"), y_test.astype("float32"))



# Define the directory where your files are located
def extract_gzip(input_dir = RAW_DIR, data_set_type=DATASET_TYPE):
    """"""
    unpacked_dir =  f"unpacked_{data_set_type}"
    # TODO: since uncompressed files are at the same heirarchy as the ain code file find a way for git 
    #       ignore to to be able to ignore other data set type folder

    if os.path.exists(unpacked_dir):
        print("Checking:", os.path.abspath(unpacked_dir))
        print("gzip directory already exist")
        return 

    # if directory does not exist 
    print("making directory")
    os.makedirs(unpacked_dir)    

    # Loop through all files in the specified directory
    # example relative path "gzip\emnist-digits-test-images-idx3-ubyte.gz"
    for filename in os.listdir(input_dir):
        # split the file name for data set type checking 
        name = filename.split("-")

        # Check if the file has a '.gz' extension
        if (filename.endswith('.gz')) and name[1] == data_set_type:
            # Construct the full path for the compressed file
            compressed_filepath = os.path.join(input_dir, filename)

            # Create the name for the new uncompressed file by removing the '.gz' extension
            uncompressed_filename = os.path.splitext(filename)[0]
            uncompressed_filepath = os.path.join(unpacked_dir, uncompressed_filename)

            # Open the compressed file and the new uncompressed file
            with gzip.open(compressed_filepath, 'rb') as f_in:
                with open(uncompressed_filepath, 'wb') as f_out:
                    f_out.write(f_in.read())

            print(f'\t- Extracted: {filename} -> {uncompressed_filename}')

extract_gzip()
load_emnist_byclass()

Checking: c:\Users\acer\Desktop\PROGRAMMING\machine_learning\NumPy-CNN-Handwritten-Digit-Recognition-from-Scratch\unpacked_byclass
gzip directory already exist


#### Helper

In [ ]:
def plot_sample(bitmap_sample):
    img = bitmap_sample.squeeze()

    f, axes =  plt.subplots(1,2)
    axes[0].imshow(np.transpose(img), cmap="gray")
    axes[1].imshow(img, cmap="gray")
    plt.show()

#### CNN implementation


In [ ]:
xtrain = np.load("datasets_npy/X_train.npy", mmap_mode='r')
ytrain = np.load("datasets_npy/Y_train.npy", mmap_mode='r')
xtest = np.load("datasets_npy/X_test.npy", mmap_mode='r')
ytest = np.load("datasets_npy/Y_test.npy", mmap_mode='r')

# reshape
xtrain = xtrain.reshape((*xtrain.shape, 1 ))
xtest = xtest.reshape((*xtest.shape, 1 ))

idx = np.random.randint(xtest.shape[0])
plot_sample(xtest[idx])
print(np.shape(xtest[idx]), ytest[idx])

class CNN:
    def __init__(self, fs=5, ps=2):
        self.filter_size = fs
        self.pool_size = ps

    def dense_layer(self, input_vector, output_size, weights=None, bias=None):
        """
        Dense (fully-connected) layer — forward pass.

        Args:
            input_vector:  1D numpy array (shape (input_size,)) OR
                        2D numpy array for a batch (shape (batch_size, input_size))
                        (If you pass a 1D vector, treat it as a single-example batch.)
            output_size:   int, number of neurons in this dense layer (e.g., number of classes)
            weights:       optional pre-initialized weight matrix (see shape note below)
            bias:          optional pre-initialized bias vector

        Returns:
            logits: raw scores (shape (output_size,) for single input or
                                (batch_size, output_size) for batch input)
        """

        # If user passed a single 1D vector, make it a batch of 1 so dot products work uniformly.
        batch_sample = input_vector
        single_example = False
        if batch_sample.ndim == 1:
            batch_sample = batch_sample.reshape((1, batch_sample.shape[0]))
            single_example = True

        batch_sample = batch_sample.astype(np.float32)

        input_size = batch_sample.shape[1]

        if weights is None:
            scale = sqrt(2 / input_size)
            weights = np.random.randn(input_size, output_size) * scale
        else:
            # If user passed weights, ensure the shape matches what we expect.
            # If mismatch, raise a clear error (or reshape if you intend to).
            # e.g. if weights.shape != (input_size, output_size): raise ValueError(...)
            if weights.shape != (input_size, output_size):
                raise ValueError("weights shape mismatch")


        if bias is None:
            bias =  np.zeros((output_size,), dtype=np.float32)
        else:
            if bias.shape != (output_size,):
                raise ValueError("bias shape mismatch")


        if isinstance(weights, np.ndarray) and isinstance(bias, np.ndarray):
            logits = batch_sample.dot(weights) + bias   # bias broadcasts across batch dim

        # If original input was 1D, return a 1D logits vector (squeeze the batch dim).
        if single_example: 
            return logits.squeeze(0)
        # Otherwise return the batched logits.
        else: 
            return logits
        


    def convolution(self, image, kernel, stride=1):
        """
        Args:
            image (sample):     2d array / bit image shape (28,28)
            kernel (filter):    2d numpy array
            stride:             no of steps a filter take

        return:
            feature map:        
        """
        H,W = image.shape
        kH, kW = kernel.shape

        feature_y = ((H - kH) // stride) + 1
        feature_x = ((W - kW) // stride) + 1

        # stores the convultion 
        feature_map = np.zeros(kH, kW)

        for h in range(feature_y):
            for w in range(feature_x):
                # a snippet of an image 
                patch = image[h * stride : h * stride + kH, w * stride : w * stride + kW]
                feature_map[h,w] = np.sum(kernel * patch)

    def reLu(self, feature_map):
        """
        returns activated feature map

        args: 
            feature_map:    2d array recieved from convulution 
        """
        # # creates a true or false mask 
        # mask = feature_map <= 0
        # # do reLu based of mask value
        # feature_map[mask] = 0
        # #return the activated feature map
        # return feature_map
    
        # more effecient way  
        return np.maximum(0, feature_map)   # -> compares the feature map element with zero (since 0 > neg num it replaces neg val with 0)

    def max_pool(self, feature_map, stride = 2, pool_size=2):
        x, y = feature_map.shape

        pool_x = ((x - pool_size) // stride) + 1
        pool_y = ((y - pool_size) // stride) + 1


        pooled_feature = np.zeroes(pool_x, pool_y)

        for h in range(pool_y):
            for w in range(pool_x):
                # snippet of the pool 
                pool = feature_map[h * stride : h * stride + pool_size, w * stride : w * stride + pool_size]
                # chose the highest value within a pool 
                pooled_feature[h,w] = np.max(pool)

        return pooled_feature
    def soft_max(Self, logits):
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Numerical stability improvement
        return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    
    def loss_function():
        pass
    
    def flatten(self, pool):
        return pool.flatten()

NameError: name 'np' is not defined